<a href="https://colab.research.google.com/github/Charmiece/Final-Project---CreditCard-Fraud/blob/Afreensiddiqa/ETL_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.1'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubun

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-03 16:04:10--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2022-04-03 16:04:10 (11.3 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://fraudcreditcard.s3.amazonaws.com/clean_data_train.csv"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("clean_data_train.csv"), sep=",", header=True, inferSchema=True)

In [ ]:
# Show DataFrame
user_data_df.show()

+--------------------+-------------+------+------+----------+---------+------------------+--------+---+
|            merchant|     category|   amt|gender| unix_time|merch_lat|        merch_long|is_fraud|age|
+--------------------+-------------+------+------+----------+---------+------------------+--------+---+
|fraud_Rippin, Kub...|     misc_net|  4.97|     F|1325376018|36.011293|        -82.048315|       0| 34|
|fraud_Heller, Gut...|  grocery_pos|107.23|     F|1325376044|49.159047|       -118.186462|       0| 44|
|fraud_Lind-Buckridge|entertainment|220.11|     M|1325376051|43.150704|       -112.154481|       0| 60|
|fraud_Kutch, Herm...|gas_transport|  45.0|     M|1325376076|47.034331|       -112.561071|       0| 55|
| fraud_Keeling-Crist|     misc_pos| 41.96|     M|1325376186|38.674999|        -78.632459|       0| 36|
|fraud_Stroman, Hu...|gas_transport| 94.63|     F|1325376248|40.653382|-76.15266700000001|       0| 61|
|fraud_Rowe-Vander...|  grocery_net| 44.54|     F|1325376282|37.